In [8]:
import random
from datetime import datetime

import geopandas as gpd
import googlemaps
import keyring
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import polyline
import seaborn as sns
from IPython.display import Markdown
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from shapely.geometry import Point, LineString

palette = sns.color_palette("colorblind", 20)


def to_geo(df, x_field='longitude', y_field='latitude'):
    geometry = [Point(xy) for xy in zip(df[x_field], df[y_field])]
    return gpd.GeoDataFrame(df, geometry=geometry)


spark = (
    SparkSession.builder
        .master("local")
        .appName("TFL Notebook")
        .config('spark.executor.memory', '8G')
        .config('spark.driver.memory', '16G')
        .config('spark.driver.maxResultSize', '10G')
        .config("spark.sql.crossJoin.enabled", "true")
        .getOrCreate()
)

BIKE_POINTS_FILE = "../data/bike-points.csv"

schema = T.StructType([
    T.StructField("idx",       T.IntegerType(), False),
    T.StructField("id",        T.IntegerType(), False),
    T.StructField("name",      T.StringType(),  False),
    T.StructField("latitude",  T.DoubleType(),  False),
    T.StructField("longitude", T.DoubleType(),  False),
    T.StructField("osgb_x",    T.DoubleType(),  False),
    T.StructField("osgb_y",    T.DoubleType(),  False),
    T.StructField("numdocks",  T.LongType(),    False),
    T.StructField("num_bikes", T.LongType(),    False),
    T.StructField("num_empty", T.LongType(),    False)
])
bike_points = spark.read.csv(BIKE_POINTS_FILE, schema=schema, header='true', mode="PERMISSIVE")
bike_points.createOrReplaceTempView("bike_points")

trips = spark.read.parquet("../data/parquet_trip")
trips.createOrReplaceTempView("trips")

df = spark.sql("""
    select (sum(duration) / 60) as duration, count(1) as trip_count, bike_id, start_year, start_month, start_day from trips
    group by bike_id, start_year, start_month, start_day
    order by trip_count desc
""")

df.createOrReplaceTempView("busy_bikes")

df.limit(10).toPandas()

,duration,trip_count,bike_id,start_year,start_month,start_day
0,636.0,32,7349,2016,1,25
1,364.0,30,13523,2016,1,29
2,564.0,30,4498,2016,2,2
3,532.0,30,9886,2016,2,3
4,236.0,30,12295,2016,2,2
5,504.0,30,13343,2016,1,29
6,626.0,28,9456,2016,2,6
7,326.0,28,9600,2016,1,28
8,588.0,28,2549,2016,1,24
9,334.0,28,11706,2017,9,1


In [11]:
spark.sql("""
    select start_ts, start_station_name, end_ts, end_station_name, duration from trips
    where bike_id = 7349 and start_year = 2016 and start_month = 1 and start_day = 25
    order by start_ts asc
""").toPandas()

,start_ts,start_station_name,end_ts,end_station_name,duration
0,2016-01-25 06:57:00,"Foley Street, Fitzrovia",2016-01-25 06:59:00,"Bolsover Street, Fitzrovia",120
1,2016-01-25 08:14:00,"Bolsover Street, Fitzrovia",2016-01-25 08:25:00,"Russell Square Station, Bloomsbury",660
2,2016-01-25 10:46:00,"Russell Square Station, Bloomsbury",2016-01-25 11:33:00,"Old Quebec Street, Marylebone",2820
3,2016-01-25 11:41:00,"Old Quebec Street, Marylebone",2016-01-25 11:49:00,"Speakers' Corner 1, Hyde Park",480
4,2016-01-25 11:50:00,"Speakers' Corner 1, Hyde Park",2016-01-25 12:01:00,"Hyde Park Corner, Hyde Park",660
5,2016-01-25 12:04:00,"Hyde Park Corner, Hyde Park",2016-01-25 12:09:00,"Albert Gate, Hyde Park",300
6,2016-01-25 12:24:00,"Albert Gate, Hyde Park",2016-01-25 12:44:00,"Black Lion Gate, Kensington Gardens",1200
7,2016-01-25 12:44:00,"Black Lion Gate, Kensington Gardens",2016-01-25 12:47:00,"Black Lion Gate, Kensington Gardens",180
8,2016-01-25 13:11:00,"Black Lion Gate, Kensington Gardens",2016-01-25 14:17:00,"Pall Mall East, West End",3960
9,2016-01-25 15:13:00,"Wellington Arch, Hyde Park",2016-01-25 15:37:00,"Queen's Gate, Kensington Gardens",1440
